# Algorithm Graph Export

Demonstrates how a tensorflow graph for a deconvolution algorithm can be built and then exported for use from other clients.

In [1]:
from flowdec import restoration as fd_restoration
import tensorflow as tf
import tempfile
import shutil
import os

Initialize the graph by specifying parameters that are constant within it.  For example, all of the following are not dynamic parts of each graph and must be set ahead of time:

- Number of dimensions for input/output data (1D vs 2D vs 3D)
    - \* This is necessary because TF FFT functions are not dynamic
- Domain in which FFT will be run (real vs complex)

In [28]:
ndims = 3
domain_type = 'complex'
algo = fd_restoration.RichardsonLucyDeconvolver(
    ndims, pad_mode='log2', real_domain_fft=(domain_type == 'real')
).initialize()

Set export directory into which the graph will be saved and clear it out, if it already exists:

In [29]:
export_dir = tempfile.mkdtemp('-graph', 'tf-')
if os.path.exists(export_dir):
    shutil.rmtree(export_dir)
export_dir

'/var/folders/vc/j01b02990c7g90mw6v_dqpsw0000gn/T/tf-ztbt44qk-graph'

Write the graph out to a plain text "pbtxt" file:

In [30]:
algo.graph.save(export_dir, save_as_text=True)

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'/var/folders/vc/j01b02990c7g90mw6v_dqpsw0000gn/T/tf-ztbt44qk-graph/saved_model.pbtxt'


As a test, reload the graph and make sure that operations within it are retrievable and runnable:

In [31]:
g = tf.Graph()
with tf.Session(graph=g) as sess:
    saver = tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], export_dir) 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:The specified SavedModel has no variables; no checkpoints were restored.


In [32]:
import numpy as np
with tf.Session(graph=g) as sess:
    dh = g.get_tensor_by_name("data:0")
    kh = g.get_tensor_by_name("kernel:0")
    ph = g.get_tensor_by_name("pad_mode:0")
    ih = g.get_tensor_by_name("niter:0")
    o = g.get_tensor_by_name("result:0")
    res = sess.run(o, feed_dict={dh: np.ones([5]*ndims), kh: np.ones([5]*ndims), ph: 'log2', ih: 10})

In [33]:
res.shape

(5, 5, 5)